# Show all detections around masks on display
Quick notebook to make plots/visual inspections similar to the last image of: https://hsc-release.mtk.nao.ac.jp/doc/index.php/known-problems-2/#hsc-link-2

Change next cell to select the patch to show and detections to keep, then run everything:

In [1]:
import numpy as np

chosenFilter = "HSC-R" 
tract = 9697
# random patch among those in my last minute run
patch = "{},{}".format(np.random.choice(range(2,7),1)[0], np.random.choice(range(2,7),1)[0])

# only plot detections in the inner region of the patch
keepInner = True
# only plot detections brighter than keepMag
keepMag = False #27

displayFrameNb = 0

## Set up

In [2]:
%matplotlib inline
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst import geom
import os

Utility function to get info from .reg files containing Andy's masks:

In [3]:
def extractBrightStarInfo(tractInfo, chosenFilter, mask_path, radMaxPix=0, verbose=False):
    f = open(mask_path, 'r')

    # boundaries; shrink BBox so largest annuli can fit in entirely
    bBox = tractInfo.getBBox()
    bBox.grow(-int(radMaxPix)-1)
    wcs = tractInfo.getWcs()

    pix_centers = []
    mags = []
    radii = []
    
    for line in f:
        if line[:6] == 'circle':
            maskinfo, comm = line.split('#')
            objid, mag = comm.split(',')
            maskinfo = maskinfo.split(',')
            radius = maskinfo[2].split('d')[0] # keep everything before the d
            circle = [float(maskinfo[0][7:]), # remove "circle("
                        float(maskinfo[1])]
            sp = geom.SpherePoint(circle[0], circle[1], geom.degrees)
            cpix = wcs.skyToPixel(sp)
            if (cpix[0] >= bBox.beginX and 
                cpix[0] < bBox.endX):
                if (cpix[1] >= bBox.beginY and 
                    cpix[1] < bBox.endY):
                    # also save magnitude (as put down by Andy)...
                    mag = float(mag[5:-2]) # remove " mag:" and the trailing \n
                    mags += [mag]
                    # ... and mask radius
                    radii += [float(radius)]
                    # if the center of the mask is inside, keep it
                    pix_centers += [cpix]
        elif line[:3] == 'box': # ignore saturation/bleed trails boxes
            pass
        else: 
            if verbose:
                # check we didn't miss anything important
                print(line)
    f.close()
    mags = np.array(mags)
    radii = np.array(radii) * 3600 # arcsec
    return pix_centers, mags, radii

In [4]:
butler = Butler("/project/mschmitz/hscRerun/mynobg")

## Show on display
Start by firing it up:

In [5]:
coadd_fn = butler.get('deepCoadd_calexp_filename', tract=tract, patch=patch, 
                      filter=chosenFilter)
print(coadd_fn)
if not os.path.exists(coadd_fn[0]):
    print('You are probably reading from parent; worth double checking!')

coadd = butler.get('deepCoadd_calexp', tract=tract, patch=patch, filter=chosenFilter)

['/project/mschmitz/hscRerun/mynobg/deepCoadd-results/HSC-R/9697/6,6/calexp-HSC-R-9697-6,6.fits']


In [6]:
display = afwDisplay.Display(displayFrameNb)
mask = coadd.getMask()
display.mtv(coadd)
display.scale("linear", "zscale")
display.setMaskTransparency(100)

Load detections and mask info:

In [7]:
src_fn = butler.get('deepCoadd_meas_filename', tract=tract, patch=patch, filter=chosenFilter)
if not os.path.exists(src_fn[0]):
    print('You are probably reading from parent; worth double checking!')
#src = butler.get('deepCoadd_ref', tract=tract, patch=patch, filter=chosenFilter)
allSrc = butler.get('deepCoadd_meas', tract=tract, patch=patch, filter=chosenFilter)
print(len(allSrc))

27336


Filter kept detections using specified criteria:

In [8]:
src = allSrc.copy()
if keepInner:
    src = src[src['detect_isPatchInner']]

if keepMag:
    photoCalib = coadd.getPhotoCalib()
    ABmags = photoCalib.instFluxToMagnitude(src, 'base_PsfFlux')
    src = src[ABmags[:,0] < keepMag]
    
print(len(src),len(allSrc))

24445 27336


In [9]:
# get sky map and WCS for this tract
skymap = butler.get("deepCoadd_skyMap", tract=tract, filter=chosenFilter)
tractInfo = skymap[tract]
wcs = tractInfo.getWcs()
pixScale = geom.radToArcsec(wcs.getPixelScale())

In [10]:
mask_path = '/datasets/hsc/BrightObjectMasks/GouldingMasksS18A/{}/BrightObjectMask-{}-{}-{}.reg'.format(
            tract,tract,patch,chosenFilter)
brightCenters, brightMags, brightRadii = extractBrightStarInfo(tractInfo,
                                                 chosenFilter, mask_path, verbose=False)

Ok, convert mask radii to pixel:

In [11]:
brightRadiiPix = brightRadii / pixScale

And plot them on display. (Note you can check these either by loading in Andy's .reg file, or comparing with the mask plane for bright objects):

In [12]:
with display.Buffering():
    for j,cpix in enumerate(brightCenters):
        display.dot("o", cpix[0], cpix[1], size=brightRadiiPix[j],
                    ctype='orange')

Determine detections within that patch and outside all bright star masks:

In [13]:
insideCents = np.array([obj.getCentroid() for obj in src])

In [14]:
outsideMask = np.ones(len(insideCents)).astype(bool)
for brightCen, pixRad in zip(brightCenters, brightRadiiPix):
    dist = np.sqrt(np.sum((insideCents - brightCen)**2, axis=1))
    outsideMask[dist < pixRad] = False

And now plot them, see if you get something like the known problem plot:

In [15]:
with display.Buffering():
    for cent in insideCents[outsideMask]:
        display.dot("o", cent[0], cent[1], 
                    size=5, ctype='green')